In [58]:
import numpy as np
from collections import defaultdict



class Node(object):
    def __init__(self, threshold=None, feature_idx=None, value=None, left=None, right=None, impurity_decrease = None):
        ## create node
        self.threshold = threshold
        self.feature_idx = feature_idx
        self.value = value 
        self.left = left
        self.right = right
        self.impurity_decrease = impurity_decrease

    def is_leaf_node(self):
        return self.value is not None

    def gini(self, class_count,n):
        """
        Calculate the Gini impurity for a list of class labels.
        """
        gini_value = 1
        for count in class_count.values():
            prob = count / n
            gini_value -= prob ** 2
        return gini_value

    def entropy(self, class_count,n):
        """
        Calculate the entropy gain for a list of class labels.
        """

        entropy_value = 0
        for count in class_count.values():
            prob = count / n
            if prob>0:
                entropy_value -= prob * np.log2(prob)
                
        return entropy_value

    def mse(self, sum_y,sum_y2,n ):
        """
        Calculate the mse for a list of regression labels.
        """
        return (sum_y2/n)-(sum_y/n)**2
    
    def errorloss(self, X, y, feature_idx, task_type, criterion='gini'):
        """
        Calculate error loss which gives best score and threshold for only one feature 
        """
        
        ## sort the x's such that easy to calculate thresholds 
        pairs = list(zip(X[:, feature_idx], y))
        sorted_pairs = sorted(pairs, key=lambda pair: pair[0])
        x_sorted, y_sorted = zip(*sorted_pairs)
        
        # convert to array for better perfomance
        x_sorted = np.array(x_sorted)
        y_sorted = np.array(y_sorted)
        
        n = len(y_sorted)
        
        # create dummys which minimises the score and helps gets the better threshold 
        best_fit = -np.inf
        best_threshold = 0
        
        
        if task_type == 'class':
            
            ## create dict such that timecomplexity is reduced 
            total_counts = defaultdict(int)
            
            for i in y_sorted:
                total_counts[i] +=1
            total_counts_main = total_counts.copy()
            left_counts = defaultdict(int)
            
            for i in range(1,n-1):
                
                left_value = y_sorted[i-1]
                left_counts[left_value] +=1
                total_counts[left_value] -=1
                right_counts = total_counts
                
                left_n  = i
                right_n = n-1-i
                
                threshold = (x_sorted[i-1] + x_sorted[i]) / 2

                if left_n == 0 or right_n == 0:
                    continue 
                    
                if x_sorted[i-1] == x_sorted[i] :
                    continue

                if criterion == 'entropy':
                    parent_loss = self.entropy(total_counts_main,n)
                    left_loss = self.entropy(left_counts,left_n)
                    right_loss = self.entropy(right_counts,right_n)
                    error_loss = parent_loss - (left_n * left_loss + right_n * right_loss)
                    if error_loss>best_fit:
                        best_fit = error_loss
                        best_threshold = threshold
                    
                elif criterion == 'gini':
                    parent_loss = self.gini(total_counts_main,n)
                    left_loss = self.gini(left_counts,left_n)
                    right_loss = self.gini(right_counts,right_n)
                    error_loss = parent_loss - (left_n * left_loss + right_n * right_loss)
                    if error_loss>best_fit:
                        best_fit = error_loss
                        best_threshold = threshold
                else:
                    raise ValueError("Invalid classicfication criteria.")
                
        elif task_type == 'regression':
            sum_total = np.sum(y_sorted)
            sum_sq_total = np.sum(np.square(y_sorted))
            sum_left = 0.0
            sum_sq_left = 0.0
            for i in range(1,n-1):
                y_val = y_sorted[i-1]
                sum_left += y_val
                sum_sq_left += np.sum(np.square(y_val))
                
                sum_right = sum_total - sum_left
                sum_sq_right = sum_sq_total - sum_sq_left
                
                left_n = i
                right_n = n-1-i
                
                parent_loss = self.mse(sum_total,sum_sq_total,n)
                left_loss = self.mse(sum_left,sum_sq_left,left_n)
                right_loss = self.mse(sum_right,sum_sq_right,right_n)
                error_loss = parent_loss - (left_n * left_loss + right_n * right_loss)
                if error_loss>best_fit:
                    best_fit = error_loss
                    best_threshold = threshold

        else:
            raise ValueError("Invalid task type.")
            
            
            
        # if error_loss>best_fit:
        #     best_fit = error_loss
        #     best_threshold = threshold

        return best_threshold,best_fit
    
    def find_best_fit(self,X,y, task_type, criterion, max_features):
        """
        Calculate best threshold for each feature and get best feature in that fit 
        """
        best_fit = -np.inf
        best_feature_idx = None
        best_threshold = 0
        
        max_features = min(max_features, X.shape[1])
        
        if max_features is None:
            max_features = int(np.sqrt(X.shape[1])) if task_type == 'class' else X.shape[1] // 3
        feature_indices = np.random.choice(X.shape[1], size=max_features, replace=False)

                    
        for i in feature_indices:
            threshold,score =  self.errorloss(X,y,i, task_type, criterion)
            if score>best_fit:
                best_fit = score
                best_feature_idx = i 
                best_threshold = threshold
        return best_feature_idx,best_threshold,best_fit
    
    def majority_class(self,y,task_type):
        """
        When stop condition hits return the value corresponding to task type 
        """
        
        if task_type == 'class':
            values,counts = np.unique(y, return_counts = True)
            return  values[np.argmax(counts)]
        elif task_type == 'regression':
            return np.mean(y)
        
    
    def fit(self,X,y,depth = 0,max_depth = 5,min_samples_split=2,task_type='class', criterion='gini', max_features = 3):
        """
        recursive function fit which creats the tree and nodes based on the best threshold and best feature 
        """
        
        
        ## first stop condition if all y are same then stop which is puritycheck
        if len(set(y))== 1:
            self.value = y[0]
            return
        
        ## second stop condition , dont go beyond  max_depth 
        if depth >= max_depth:
            self.value = self.majority_class(y,task_type)
            return
        
        ## if the split has sample size less than minum minimum sample split then stop
        if len(y) < min_samples_split:
            self.value = self.majority_class(y,task_type)
            return 
        
        ## get the best feature and threshold from find best fit 
        best_feature_idx,best_threshold,best_fit = self.find_best_fit(X,y,task_type, criterion,max_features)
        
        ## stop confition when there are no best features 
        if best_feature_idx is None:
            self.value = self.majority_class(y, task_type)
            return
        
        ## save asplit info for prediction 
        
        self.threshold = best_threshold
        self.feature_idx = best_feature_idx
        self.impurity_decrease = best_fit
        
        
        ## create next child nodes 
        print(f"Depth: {depth}, Feature: {best_feature_idx}, Threshold: {best_threshold}")
        
        mask_left = X[:, best_feature_idx] <= best_threshold
        X_left = X[mask_left]
        y_left = y[mask_left]
        self.left = Node()
        self.left.fit(X_left,y_left,depth+1,max_depth,min_samples_split,task_type, criterion,max_features)
        
        mask_right = X[:, best_feature_idx] > best_threshold
        X_right = X[mask_right]
        y_right = y[mask_right]
        
        self.right = Node()
        self.right.fit(X_right,y_right,depth+1,max_depth,min_samples_split,task_type, criterion,max_features)
        
    def predict_single(self,x):
        if self.is_leaf_node():
            return self.value
        else :
            if x[self.feature_idx] <= self.threshold:
                return self.left.predict_single(x)
            else:
               
                return self.right.predict_single(x)
    def predict(self, x):
        return [self.predict_single(row) for row in x.tolist()]
    





class BaggedTrees(object):
    def __init__(self, n_estimators = 10, max_depth = 5, min_samples_split = 2, task_type = 'class', criterion = 'gini', max_features = 3, random_state = 3):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.task_type = task_type
        self.max_features = max_features
        self.criterion = criterion
        self.random_state = random_state
        self.trees = []
        
    def fit(self,X,y) : 
        for i in range(0, self.n_estimators):
            np.random.seed(self.random_state + i)
            indices = np.random.choice(len(X), size=len(X), replace=True)
            X_sample = X[indices]
            y_sample = y[indices]
            
            if self.max_features > X_sample.shape[1]:
                max_features = X_sample.shape[1]
            else : 
                max_features = self.max_features
                
            tree = Node()
            tree.fit(X_sample,y_sample, depth = 0 ,max_depth = self.max_depth, min_samples_split = self.min_samples_split, task_type = self.task_type, criterion = self.criterion, max_features = max_features )
            self.trees.append(tree)
            
    def predict_single(self,x):
        
        if self.task_type == 'class':
            predictions = 0
            for i in self.trees:
                predictions_value = i.predict_single(x)
                if predictions_value == 1 :
                    predictions += 1
            prediction_final = 0 if predictions/self.n_estimators<=0.5 else 1
            return prediction_final
        
        else:
            predictions = []
            for i in self.trees:
                predictions_value = i.predict_single(x)
                predictions.append(predictions_value)
            return np.mean(predictions)
        
    def predict(self, x):
        return [self.predict_single(row) for row in x.tolist()] 
                    
                
                    
                    
            

In [59]:
X = np.array([
    [2, 3,2,3],
    [1, 5,1,5],
    [8, 7,8,7],
    [9, 6,9,6],
])

y = np.array([0, 0, 1, 1])

tree = Node()
tree.fit(X, y, max_depth=3, task_type='class', criterion='gini')

preds = tree.predict(X)
print(preds)


Depth: 0, Feature: 1, Threshold: 5.5
[np.int64(0), np.int64(0), np.int64(1), np.int64(1)]


In [60]:
RandomF = BaggedTrees().fit(X, y)

Depth: 0, Feature: 1, Threshold: 5.5
Depth: 0, Feature: 2, Threshold: 4.5
Depth: 0, Feature: 1, Threshold: 5.5
Depth: 0, Feature: 0, Threshold: 5.0
Depth: 0, Feature: 2, Threshold: 5.0
Depth: 0, Feature: 0, Threshold: 1.5
Depth: 0, Feature: 3, Threshold: 6.0
Depth: 0, Feature: 2, Threshold: 1.5
Depth: 0, Feature: 2, Threshold: 5.5
Depth: 0, Feature: 0, Threshold: 4.5


In [61]:
preds = tree.predict(X)
print(preds)

[np.int64(0), np.int64(0), np.int64(1), np.int64(1)]
